In [42]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

/nfshome/htseng/courses/CSE142/demo/amdahlslaw/notebook.py:229: SyntaxWarning: invalid escape sequence '\s'
  if show is not None and len(show) == 2:
/nfshome/htseng/courses/CSE142/demo/amdahlslaw/notebook.py:229: SyntaxWarning: invalid escape sequence '\s'
  if show is not None and len(show) == 2:
/nfshome/htseng/courses/CSE142/demo/amdahlslaw/notebook.py:229: SyntaxWarning: invalid escape sequence '\}'
  if show is not None and len(show) == 2:


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Amdahl's Law

## The starting example -- sorting

In [ ]:
render_code("./gpusort/main.cu", show="main")

## Where is the most time critical part of my program?

In [ ]:
! cd gpusort; make clean; make
! echo "File on H.D.D.; Sorting on CPU"
! cd ./gpusort; echo "ET,FileInput,CPU_Kernel,GPU_Kernel,Host2GPU,GPU2Host" > sort.csv; source ./run_CPU 2>> sort.csv

In [ ]:
display_df_mono(render_csv("./gpusort/sort.csv", columns=["ET","FileInput","CPU_Kernel"]))

In [ ]:
! lscpu

### Use gprof to figure out the timing breakdown

In [ ]:
! cd gpusort; make clean; make EXTRA_FLAGS=-pg 
! cd ./gpusort; source ./run_CPU

In [ ]:
! cd gpusort; gprof ./hybridsort_cpu ./gmon.out

## Amdahl's Law -- optimizating is a moving target

In [ ]:
render_code("./gpusort/main.cu", lang="c++", show="bitonic_sort")

In [ ]:
! nvidia-smi -a

In [ ]:
! cd gpusort; make clean; make
# ! ssh htseng@azelf "source ./courses/CS203/demo/amdahlslaw/gpusort/run_CPU"
! echo "File on H.D.D.; Sorting on GPU"
! cd gpusort; source ./run 2>> sort.csv

In [ ]:
display_df_mono(render_csv("./gpusort/sort.csv"))

In [ ]:
! echo "File on S.S.D.; Sorting on GPU"
! cd gpusort; source ./run_SSD 2>> sort.csv

In [ ]:
display_df_mono(render_csv("./gpusort/sort.csv"))

## Amdahl's Law on parallel programming

In [ ]:
! cd vmul; make clean; make
! echo "THREADS,CPUTIME,HOST2GPU,GPUTIME,GPU2HOST" > ./vmul/vmul.csv
! echo "CPU based vul"
! time ./vmul/vmul 33554432 1 0 30 2>> ./vmul/vmul.csv
! echo "GPU based vul"
### i stands for "How many iterations each thread performs 
### -- the larger the number, the fewer the parallelism
! for i in 1 2 4 8 16 32 64 128 256 512 1024 2048 4096 8192; do time ./vmul/vmul 33554432 $i 1 30 2>> ./vmul/vmul.csv ; done

In [ ]:
df = render_csv("./vmul/vmul.csv")
df["TOTAL"] = df["CPUTIME"] + df["HOST2GPU"] + df["GPUTIME"] + df["GPU2HOST"]
df = df.sort_values(by=["THREADS"], ascending=True)
display_df_mono(df)
plotPE(df=df, lines=True, what=[ ('THREADS', "TOTAL"), ('THREADS', "GPUTIME")], columns=2)

# Choose the "right" metrics

## Throughput and Latency

### SSD v.s. HDD

You may use to hdparm (need root permission to execute). The /dev/sda on this machine is an SATA SSD that has around 450-500MB/sec bandwidth. The /dev/md0 is a RAID contains two H.D.Ds in RAID-0 configuration that also achieves 450-500MB/sec bandwidth. Let's examine the bandwidth using the following command.

In [ ]:
from IPython.display import IFrame
IFrame("https://hub.escalab.org:8000/user/htseng/terminals/1", width="100%", height="400")

Now, let's revisit the optimized gpusort on this machine with different array size...

In [ ]:
! echo "Configuration,Size,ET,FileInput,CPU_Kernel,GPU_Kernel,Host2GPU,GPU2Host" > sort_small.csv
! echo "File on H.D.D"
! cd gpusort; source ./run_small 512 2>> ../sort_small.csv
! echo "File on S.S.D"
! cd gpusort; source ./run_small_SSD 512 2>> ../sort_small.csv
! echo "File on H.D.D"
! cd gpusort; source ./run_small 32768 2>> ../sort_small.csv
! echo "File on S.S.D"
! cd gpusort; source ./run_small_SSD 32768 2>> ../sort_small.csv
! echo "File on H.D.D"
! cd gpusort; source ./run_small 262144 2>> ../sort_small.csv
! echo "File on S.S.D"
! cd gpusort; source ./run_small_SSD 262144 2>> ../sort_small.csv
display_df_mono((render_csv("sort_small.csv")))

What can we observe here?

## FLOPs

In [44]:
! cd metrics; make; echo "size,GLOPS,GPU_time" > flops.csv

make: Nothing to be done for 'all'.


In [22]:
! cd ./metrics; time ./cpumm 2048 512

Data type size: 4

Time: 4433.704 ms

Throughput: 3.87 GFLOPS


real	0m4.557s
user	0m4.540s
sys	0m0.016s


In [45]:
! cd ./metrics; time ./cudamm 2048 1 2>> flops.csv 

Data Type Size: 4
Time elapsed on matrix multiplication of 2048x2048 . 2048x2048 on GPU: 20.699425 ms.


Throughput: 829.97 GFLOPS


real	0m0.471s
user	0m0.123s
sys	0m0.344s


In [46]:
! cd ./metrics; time ./cudamm 4096 1 2>> flops.csv 

Data Type Size: 4
Time elapsed on matrix multiplication of 4096x4096 . 4096x4096 on GPU: 160.060760 ms.


Throughput: 858.67 GFLOPS


real	0m0.894s
user	0m0.503s
sys	0m0.387s


In [58]:
! cd metrics; time ./cudamm 8192 1 2>> flops.csv 

Data Type Size: 4
Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 816.527283 ms.


Throughput: 1346.57 GFLOPS


real	0m2.610s
user	0m2.127s
sys	0m0.479s


In [57]:
! ssh htseng@lapras "time /nfshome/htseng/courses/CSE142/demo/amdahlslaw/metrics/cudamm 8192 1 2>> /nfshome/htseng/courses/CSE142/demo/amdahlslaw/metrics/flops.csv "

Data Type Size: 4
Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 347.209137 ms.


Throughput: 3166.71 GFLOPS


real	0m1.991s
user	0m1.626s
sys	0m0.359s


In [62]:
FLOPS_Ratio = 781.863586/348.333954
print(FLOPS_Ratio)

2.2445804579819977


In [61]:
Speedup = 2.610/1.991
print(Speedup)

1.3108990457056755


In [ ]:
! cd metrics; time ./cudamm_double 2048 1

In [ ]:
! cd metrics;  ./cudamm_double 2048 0

In [ ]:
! cd metrics; time ./cpumm_double 2048 512

In [ ]:
! cd metrics;  time ./cudamm 2048 0